In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
input_size = 9
hidden_size = 16
num_layers = 2

lstm = nn.LSTM(input_size, hidden_size, num_layers)
lstm

LSTM(9, 16, num_layers=2)

In [3]:
??nn.LSTM

Init signature: nn.LSTM(*args, **kwargs)
Source:        
class LSTM(RNNBase):
    r"""__init__(input_size,hidden_size,num_layers=1,bias=True,batch_first=False,dropout=0.0,bidirectional=False,proj_size=0,device=None,dtype=None)

    Apply a multi-layer long short-term memory (LSTM) RNN to an input sequence.
    For each element in the input sequence, each layer computes the following
    function:

    .. math::
        \begin{array}{ll} \\
            i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
            f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
            g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
            o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
            c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
            h_t = o_t \odot \tanh(c_t) \\
        \end{array}

    where :math:`h_t` is the hidden state at time `t`, :math:`c_t` is the cell
    state at time `t`, :math:`x_t` is the input at time `t`,

In [5]:
seq_length = 5
batch_size = 2

x = torch.randn(seq_length, batch_size, input_size)

H = torch.zeros(num_layers, batch_size, hidden_size)
C = torch.zeros(num_layers, batch_size, hidden_size)

hidden_inputs = (H, C)

y, h = lstm(x, hidden_inputs)
print(f"Input shape: {list(x.shape)}")
print(f"Hidden shape: {list(h[0].shape)}")
print(f"Cell shape: {list(h[1].shape)}")
print(f"Output shape: {list(y.shape)}")

Input shape: [5, 2, 9]
Hidden shape: [2, 2, 16]
Cell shape: [2, 2, 16]
Output shape: [5, 2, 16]


In [6]:
for p in lstm.named_parameters():
    if "weight" in p[0]:
        print(f"{p[0]} has size {p[1].shape}")

weight_ih_l0 has size torch.Size([64, 9])
weight_hh_l0 has size torch.Size([64, 16])
weight_ih_l1 has size torch.Size([64, 16])
weight_hh_l1 has size torch.Size([64, 16])


In [7]:
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.out = nn.Linear(hidden_size, 1)

    def forward(self, x, debug=False):
        if debug:
            print(f"Input Size: {x.shape}")
        x, h = self.lstm(x)
        if debug:
            print(f"Hidden shape: {list(h[0].shape)}")
            print(f"Cell shape: {list(h[1].shape)}")
            print(f"LSTMout shape: {list(x.shape)}")
        x = self.out(x)
        if debug:
            print(f"FinalOut shape: {list(x.shape)}")
        return x, h

In [8]:
lstm = LSTMNet(input_size, hidden_size, num_layers)
lstm

LSTMNet(
  (lstm): LSTM(9, 16, num_layers=2)
  (out): Linear(in_features=16, out_features=1, bias=True)
)

In [ ]:
x = torch.randn(seq_length, batch_size, input_size)
y_pred, h = lstm(x)
print(y_pred.shape)
print(h[0].shape)
print(h[1].shape)

lossfunc = nn.MSELoss()
loss = lossfunc(y_pred, torch.randn(seq_length, batch_size, 1))
#loss = lossfunc(y_pred[-1], torch.randn(batch_size, 1)) # is also valid
loss


torch.Size([5, 2, 1])
torch.Size([2, 2, 16])
torch.Size([2, 2, 16])


tensor(0.2710, grad_fn=<MseLossBackward0>)

In [14]:
class GruNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size, hidden_size, num_layers)
        self.out = nn.Linear(hidden_size, 1)

    def forward(self, x, debug=False):
        if debug:
            print(f"Input Size: {x.shape}")
        x, h = self.gru(x)
        if debug:
            print(f"Hidden shape: {list(h[0].shape)}")
            print(f"Cell shape: {list(h[1].shape)}")
            print(f"GruOut shape: {list(x.shape)}")
        x = self.out(x)
        if debug:
            print(f"FinalOut shape: {list(x.shape)}")
        return x, h

In [15]:
gru = GruNet(input_size, hidden_size, num_layers)
x = torch.randn(seq_length, batch_size, input_size)
y_pred, h = gru(x)
print(y_pred.shape)
print(h[0].shape)
print(h[1].shape)

lossfunc = nn.MSELoss()
loss = lossfunc(y_pred, torch.randn(seq_length, batch_size, 1))
#loss = lossfunc(y_pred[-1], torch.randn(batch_size, 1)) # is also valid
loss


torch.Size([5, 2, 1])
torch.Size([2, 16])
torch.Size([2, 16])


tensor(1.5894, grad_fn=<MseLossBackward0>)

In [16]:
for p in gru.named_parameters():
    if "weight" in p[0]:
        print(f"{p[0]} has size {p[1].shape}")

gru.weight_ih_l0 has size torch.Size([48, 9])
gru.weight_hh_l0 has size torch.Size([48, 16])
gru.weight_ih_l1 has size torch.Size([48, 16])
gru.weight_hh_l1 has size torch.Size([48, 16])
out.weight has size torch.Size([1, 16])
